# Perfilamiento sobre algoritmo Simplex

### 1. Medición de tiempo

Características de la instancia que utilizamos para el perfilamiento

In [1]:
%%bash
lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              8
On-line CPU(s) list: 0-7
Thread(s) per core:  2
Core(s) per socket:  4
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               85
Model name:          Intel(R) Xeon(R) Platinum 8175M CPU @ 2.50GHz
Stepping:            4
CPU MHz:             3100.034
BogoMIPS:            4999.99
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            1024K
L3 cache:            33792K
NUMA node0 CPU(s):   0-7
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid aperfmperf tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt tsc_deadline_timer aes xsave avx f16c rdrand hy

In [2]:
%%bash
sudo lshw -C memory

  *-memory
       description: System memory
       physical id: 0
       size: 30GiB


In [3]:
import math
import time
from versiones import Simplex_v0
import numpy as np
from pytest import approx
from scipy.optimize import linprog

### Módulo time

Ejemplo Maximización para evualuar tiempo de ejecución

In [4]:
c = [3, 5]
b = [4, 12, 18]
A = [[1,  0],
    [0,  2],
    [3, 2]]

start_time = time.time()
problema = Simplex_v0.Simplex(c,A,b,problem='Max')
method_result, opt, status = problema.solve()
end_time = time.time()
secs = end_time-start_time
print("Simplex algorithm tomó",secs,"segundos" )

Optimization completed successfully !
Solution for x vector:
[2.0, 6.0]
Optimal value:
-36.0
Simplex algorithm tomó 0.001188039779663086 segundos


Con Scipy

In [5]:
c_scipu=[-3, -5]

start_time = time.time()
opt = linprog(c=c_scipu, A_ub=A, b_ub=b,
              method="simplex")
end_time = time.time()
secs = end_time-start_time
print("Scipy  tomó",secs,"segundos" )

Scipy  tomó 0.00495147705078125 segundos


Obs. En esta primera parte nos damos cuenta que el algoritmo implementado es un poco más rápido que el de Scipy.

### Comando Magic %time

In [6]:
%time problema.solve()

Optimization completed successfully !
Solution for x vector:
[0, 0]
Optimal value:
0
CPU times: user 550 µs, sys: 116 µs, total: 666 µs
Wall time: 496 µs


([0, 0], 0, 0)

### Cprofile

Para poder visualizar en que secciones de código se tarda más

In [7]:
import cProfile

In [8]:
cprof = cProfile.Profile()
cprof.enable()
problema = Simplex_v0.Simplex(c,A,b,problem='Max')
method_result, opt, status = problema.solve()
cprof.disable()
cprof.print_stats(sort='cumtime')

Optimization completed successfully !
Solution for x vector:
[2.0, 6.0]
Optimal value:
-36.0
         476 function calls in 0.001 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        3    0.000    0.000    0.001    0.000 interactiveshell.py:3288(run_code)
        3    0.000    0.000    0.001    0.000 {built-in method builtins.exec}
        1    0.000    0.000    0.001    0.001 <ipython-input-8-7a7c5a5f5a9a>:4(<module>)
        1    0.000    0.000    0.001    0.001 Simplex_v0.py:44(solve)
       16    0.000    0.000    0.000    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
        5    0.000    0.000    0.000    0.000 {built-in method builtins.print}
       10    0.000    0.000    0.000    0.000 iostream.py:384(write)
        6    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(dot)
       11    0.000    0.000    0.000    0.000 iostream.py:195(schedule)
        4    0.00

In [9]:
import pstats

cprof.dump_stats("Simplex_stats")

In [10]:
p_simplex_stats = pstats.Stats("Simplex_stats")
print(p_simplex_stats.sort_stats("cumulative").print_stats(10))

Sat May  8 19:06:52 2021    Simplex_stats

         476 function calls in 0.001 seconds

   Ordered by: cumulative time
   List reduced from 85 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        3    0.000    0.000    0.001    0.000 /usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3288(run_code)
        3    0.000    0.000    0.001    0.000 {built-in method builtins.exec}
        1    0.000    0.000    0.001    0.001 <ipython-input-8-7a7c5a5f5a9a>:4(<module>)
        1    0.000    0.000    0.001    0.001 /shared_volume/practica-2-segunda-parte-yefovar/perfilamiento/versiones/Simplex_v0.py:44(solve)
       16    0.000    0.000    0.000    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
        5    0.000    0.000    0.000    0.000 {built-in method builtins.print}
       10    0.000    0.000    0.000    0.000 /usr/local/lib/python3.6/dist-packages/ipykernel/iostream.py:384(wri

In [11]:
print(p_simplex_stats.sort_stats("cumulative").print_stats("solve|module"))

Sat May  8 19:06:52 2021    Simplex_stats

         476 function calls in 0.001 seconds

   Ordered by: cumulative time
   List reduced from 85 to 7 due to restriction <'solve|module'>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.001    0.001 <ipython-input-8-7a7c5a5f5a9a>:4(<module>)
        1    0.000    0.000    0.001    0.001 /shared_volume/practica-2-segunda-parte-yefovar/perfilamiento/versiones/Simplex_v0.py:44(solve)
        4    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(solve)
        4    0.000    0.000    0.000    0.000 /usr/local/lib/python3.6/dist-packages/numpy/linalg/linalg.py:314(solve)
        1    0.000    0.000    0.000    0.000 <ipython-input-8-7a7c5a5f5a9a>:3(<module>)
        1    0.000    0.000    0.000    0.000 <ipython-input-8-7a7c5a5f5a9a>:5(<module>)
        4    0.000    0.000    0.000    0.000 /usr/local/lib/python3.6/dist-packages/numpy/linalg/linalg.py:310(_solve_dispatche

In [12]:
#numero de llamadas a funciones primitivas
print(p_simplex_stats.prim_calls)

476


In [13]:
p_simplex_stats.strip_dirs().sort_stats("cumulative").print_callers(10)

   Ordered by: cumulative time
   List reduced from 85 to 10 due to restriction <10>

Function                                                                 was called by...
                                                                             ncalls  tottime  cumtime
interactiveshell.py:3288(run_code)                                       <- 
{built-in method builtins.exec}                                          <-       3    0.000    0.001  interactiveshell.py:3288(run_code)
<ipython-input-8-7a7c5a5f5a9a>:4(<module>)                               <-       1    0.000    0.001  {built-in method builtins.exec}
Simplex_v0.py:44(solve)                                                  <-       1    0.000    0.001  <ipython-input-8-7a7c5a5f5a9a>:4(<module>)
{built-in method numpy.core._multiarray_umath.implement_array_function}  <-       1    0.000    0.000  <__array_function__ internals>:2(concatenate)
                                                                             

In [16]:
p_simplex_stats.strip_dirs().sort_stats("cumulative").print_callees("print|dot|ndim|solve")

   Ordered by: cumulative time
   List reduced from 85 to 10 due to restriction <'print|dot|ndim|solve'>

Function                                 called...
                                             ncalls  tottime  cumtime
Simplex_v0.py:44(solve)                  ->       1    0.000    0.000  <__array_function__ internals>:2(copy)
                                                  6    0.000    0.000  <__array_function__ internals>:2(dot)
                                                  2    0.000    0.000  <__array_function__ internals>:2(size)
                                                  4    0.000    0.000  <__array_function__ internals>:2(solve)
                                                  1    0.000    0.000  index_tricks.py:316(__getitem__)
                                                  2    0.000    0.000  twodim_base.py:152(eye)
                                                  7    0.000    0.000  {built-in method builtins.len}
                                

Obs. Se observa que las partes del código que tienen un mayor número de llamadas son

* **print**- El cual se utiliza al final del algoritmo para mostrar los valores óptimos 
* **dot**(producto punto)
* **dim** y **size** que se ocupan al principio del algoritmo para saber la longuitud de las matrices a optimizar
* **solve**- Este es el que tiene mayor número de llamadas

### Lineprofiler

Para saber línea por línea en que parte nuestro algoritmo se está tardano más en ejecutar

In [14]:
import line_profiler

problema = Simplex_v0.Simplex(c,A,b,problem='Max')
line_prof = line_profiler.LineProfiler()
print(line_prof(problema.solve)())

Optimization completed successfully !
Solution for x vector:
[2.0, 6.0]
Optimal value:
-36.0
([2.0, 6.0], -36.0, 0)


In [15]:
print(line_prof.print_stats())

Timer unit: 1e-06 s

Total time: 0.000956 s
File: /shared_volume/practica-2-segunda-parte-yefovar/perfilamiento/versiones/Simplex_v0.py
Function: solve at line 44

Line #      Hits         Time  Per Hit   % Time  Line Contents
    44                                               def solve(self):
    45                                                   
    46                                                   """
    47                                                   Solves the simplex algorithm. 
    48                                                   Returns
    49                                                   -------
    50                                                   :solution: Numpy array with solution
    51                                                   """
    52         1          2.0      2.0      0.2          problem = self.problem
    53         1          2.0      2.0      0.2          c_N = self.c
    54         1          2.0      2.0      0.2          A = 

De acuerdo a las estadísticas resultantes con line profiler se tienen 7 líneas en el código que tienen un porcentaje elevado de tiempo en ejecución:
* Línea 64 - `A = np.c_[A,identity_A]` que es la parte que construye la matriz completa A al agregar la identidad por las variables de holgura
* Línea 81 - `lista.append (-lambda_ + np.dot(nu, A[:, N_list_idx[i]]))` para crear la lista de las lambas a evaluar en el método
* Línea 88 - ` d = np.linalg.solve(B, A[:,idx_x_N]) ` para la solución del problema de ecuaciones
* Línea 97 -  `if np.isnan(lista2).all() == True:`para evular una solución *ounbounded*
* Línea 117 - `nu = np.linalg.solve(B.T, c_B)`solución del problema de ecuaciones
* Línea 137 - `print("Optimization completed successfully !") `para imprimir si la optimización fue correcta


Respecto a las demás líneas, lo que decidimos mejorar fue la línea 63 con otra función de Pyhton: `np.hstack`  y evaluar los resultados.

**Modificando la línea 63 con la funcion `np.hstack`**

In [16]:
import line_profiler
from versiones import Simplex_v0_1

c = [3, 5]
b = [4, 12, 18]
A = [[1,  0],
    [0,  2],
    [3, 2]]

problema = Simplex_v0_1.Simplex(c,A,b,problem='Max')
line_prof = line_profiler.LineProfiler()
print(line_prof(problema.solve)())

Optimization completed successfully !
Solution for x vector:
[2.0, 6.0]
Optimal value:
-36.0
([2.0, 6.0], -36.0, 0)


In [17]:
print(line_prof.print_stats())

Timer unit: 1e-06 s

Total time: 0.000925 s
File: /shared_volume/practica-2-segunda-parte-yefovar/perfilamiento/versiones/Simplex_v0_1.py
Function: solve at line 44

Line #      Hits         Time  Per Hit   % Time  Line Contents
    44                                               def solve(self):
    45                                                   
    46                                                   """
    47                                                   Solves the simplex algorithm. 
    48                                                   Returns
    49                                                   -------
    50                                                   :solution: Numpy array with solution
    51                                                   """
    52         1          3.0      3.0      0.3          problem = self.problem
    53         1          1.0      1.0      0.1          c_N = self.c
    54         1          1.0      1.0      0.1          A 

Como se puede observar en las estadísticas de arriba la línea 64 si mejora bastante en las 3 columnas : Time, Per Hit y % Time. Con lo cual se logró reducir el % de tiempo de ejecución, se pasó de 11  a  5.9.

Donde la línea que ocupa más recursos es la 137, únicamente se llama una vez a esa parte del código pero sabemos que los print siempre son computacionalmente costosos, por lo cual podríamos considerar eliminarlo o solo realizar la impresión si el usuario lo desea con un **verbose**. 

**Agregando un `verbose` para los prints de la solución**

In [25]:
import line_profiler
import Simplex

c = [3, 5]
b = [4, 12, 18]
A = [[1,  0],
    [0,  2],
    [3, 2]]

problema = Simplex.Simplex(c,A,b,problem='Max')
line_prof = line_profiler.LineProfiler()
print(line_prof(problema.solve)())

Solution for x vector, optimization value and status:
([2.0, 6.0], -36.0, 0)


In [19]:
#Con verbose=True
problema = Simplex.Simplex(c,A,b,problem='Max', verbose=True)
line_prof = line_profiler.LineProfiler()
print(line_prof(problema.solve)())

Optimization completed successfully !
Solution for x vector:
[2.0, 6.0]
Optimal value:
-36.0
Solution for x vector, optimization value and status:
([2.0, 6.0], -36.0, 0)


Evaluado en `line_profiler` con las modificaciones

In [31]:
problema = Simplex.Simplex(c,A,b,problem='Max')
line_prof = line_profiler.LineProfiler()
print(line_prof(problema.solve)())

Solution for x vector, optimization value and status:
([2.0, 6.0], -36.0, 0)


In [32]:
print(line_prof.print_stats())

Timer unit: 1e-06 s

Total time: 0.000949 s
File: /usr/local/lib/python3.6/dist-packages/Simplex/__init__.py
Function: solve at line 45

Line #      Hits         Time  Per Hit   % Time  Line Contents
    45                                               def solve(self):
    46                                                   
    47                                                   """
    48                                                   Solves the simplex algorithm. 
    49                                                   Returns
    50                                                   -------
    51                                                   :solution: Numpy array with solution
    52                                           
    53                                                   """
    54         1          2.0      2.0      0.2          problem = self.problem
    55         1          1.0      1.0      0.1          verbose = self.verbose
    56         1          1.

De acuerdo a los últimos estadísticos de con la nueva versión del paquete logramos reducir el tiempo que por default acumulaba el print anterior de la solución, ahora le dejamos al usuarios que si desea ver los print's en la ejecución lo haga poniendo el parámetro `verbose=True`, de otra manera podría acceder a los valores simplemente imprimiendo : x_result, opt y status.

**Evalaución del tiempo en segundos con `time` con el último algoritmo:**

In [33]:
start_time = time.time()
problema = Simplex.Simplex(c,A,b,problem='Max')
method_result, opt, status = problema.solve()
end_time = time.time()
secs = end_time-start_time
print("Simplex algorithm tomó",secs,"segundos" )

Solution for x vector, optimization value and status:
Simplex algorithm tomó 0.0009491443634033203 segundos


In [35]:
#Versión anterior v0
start_time = time.time()
problema = Simplex_v0.Simplex(c,A,b,problem='Max')
method_result, opt, status = problema.solve()
end_time = time.time()
secs = end_time-start_time
print("Simplex algorithm tomó",secs,"segundos" )

Optimization completed successfully !
Solution for x vector:
[2.0, 6.0]
Optimal value:
-36.0
Simplex algorithm tomó 0.0011134147644042969 segundos


Conclusión:

Si logramos mejorar el tiempo de ejecución del algoritmo y realmente comparado con nuestro benchamark de "Scipy" tiene un buen desempeño en tiempo.